In [2]:
import pandas as pd
import unicodedata
import re
import numpy as np

# For classifier input

In [65]:
df = pd.read_csv("results/validation_set.csv")
#test.to_csv("results/test_set.csv")
df.head()

,Unnamed: 0,body,sex,NEL,subreddit,text,processed,len,processed_sub100000,processed_subsmaller
0,40053,It's Ephebophilia. Pedophilia is what Schiff...,female,Q235349,The_Donald,it s ephebophilia . pedophilia is what schiff ...,"['it', 's', 'ephebophilia', '.', 'pedophilia',...",14.0,"['it', 's', 'unk', '.', 'pedophilia', 'is', 'w...","['it', 's', 'unk', '.', 'pedophilia', 'is', 'w..."
1,5714224,[NAME] didn't switch parties. [NAME] declared...,male,Q1714165,politics,name didn t switch parties . name declared na...,"['', 'name', 'didn', 't', 'switch', 'parties',...",62.0,"['', 'name', 'didn', 't', 'switch', 'parties',...","['', 'name', 'didn', 't', 'switch', 'parties',..."
2,3974237,What if we just attack [NAME]’s Dem challenger...,male,Q22212,politics,what if we just attack name s dem challenger f...,"['what', 'if', 'we', 'just', 'attack', 'name',...",16.0,"['what', 'if', 'we', 'just', 'attack', 'name',...","['what', 'if', 'we', 'just', 'attack', 'name',..."
3,5437084,"I also wonder if [NAME] knew about it, or peop...",male,Q632321,politics,i also wonder if name knew about it or people ...,"['i', 'also', 'wonder', 'if', 'name', 'knew', ...",20.0,"['i', 'also', 'wonder', 'if', 'name', 'knew', ...","['i', 'also', 'wonder', 'if', 'name', 'knew', ..."
4,1756989,Superiority complex.And then some. Are you fa...,female,Q214475,The_Donald,superiority complex . and then some . are you ...,"['superiority', 'complex', '.', 'and', 'then',...",91.0,"['superiority', 'complex', '.', 'and', 'then',...","['superiority', 'complex', '.', 'and', 'then',..."


In [68]:
df.rename({'Unnamed: 0': 'index'}, axis=1, inplace=True)
clean = df[['index','body','sex']]
clean.to_csv("valid_set_clean.tsv",sep='\t',index=False, header=None)

In [69]:
clean.head()

,index,body,sex
0,40053,It's Ephebophilia. Pedophilia is what Schiff...,female
1,5714224,[NAME] didn't switch parties. [NAME] declared...,male
2,3974237,What if we just attack [NAME]’s Dem challenger...,male
3,5437084,"I also wonder if [NAME] knew about it, or peop...",male
4,1756989,Superiority complex.And then some. Are you fa...,female


In [70]:
df = pd.read_csv("results/test_set.csv")
df.rename({'Unnamed: 0': 'index'}, axis=1, inplace=True)
clean = df[['index','body','sex', 'subreddit']]
clean.to_csv("test_set_clean.tsv",sep='\t',index=False, header=None)

# Cross-partisan Nominal Bias comparison

In [5]:
# Create data for name use cross-partisan comparison

df = pd.read_csv("all_comments.csv")

sub = df[['subreddit','sex', 'given_name_used',
       'family_name_used', 'full_name_used', 'nickname_used']]

left = ['Liberal', 'SocialDemocracy', 'socialism', 'alltheleft', 'neoliberal', 'democrats']
right = ['Libertarian', 'Conservative', 'Republican']
alt_right = ['The_Donald']
maps = {}
for sr in left:
    maps[sr] = 'left'
for sr in right:
    maps[sr] = 'right'
maps['The_Donald'] = 'alt_right'
sub['group'] = sub['subreddit'].map(maps)
sub.dropna(subset=['group'], inplace=True)

#sub.melt(id_vars=['sex','group'], value_vars=['given_name_used','family_name_used', 'full_name_used', 'nickname_used'], var_name='Name_used')
sub[['given_name_used','family_name_used', 'full_name_used', 'nickname_used']].idxmax(axis=1)
sub['name'] = sub[['given_name_used','family_name_used', 'full_name_used', 'nickname_used']].astype(int).idxmax(axis=1)

sub = sub[sub['sex'].isin(['male', 'female'])]
sub.to_csv("names_used.csv")

# Create data splits

In [3]:
df = pd.read_csv("results/total.csv")
df = df[df.sex.isin(['male','female'])]
print(df.shape)
print(df.shape[0]*0.7)
print(df.sex.value_counts(normalize=True))

C:\Users\vanki\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(8111768, 9)
5678237.6
male      0.880532
female    0.119468
Name: sex, dtype: float64


In [4]:
df.NEL.value_counts()[0].sum()

3208707

In [5]:
5639925/8111768

0.6952769112726104

In [6]:
entities = [0,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21, 22, 23, 25, 26, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]

In [7]:
df.NEL.value_counts()[entities].sum()

5639925

In [8]:
sub = df.groupby('NEL').sex.value_counts().sort_values(ascending=False)[entities]
sub.unstack(level=0).sum(axis=1) / sub.unstack(level=0).sum().sum()

sex
female    0.119408
male      0.880592
dtype: float64

In [13]:
ent_ids= [i[0] for i in sub.index]
train = df[df['NEL'].isin(ent_ids)]
test_valid = df[~df['NEL'].isin(ent_ids)]
train.to_csv("results/train_set.csv", index=False)
shuffled = test_valid.sample(frac=1)
valid = shuffled.iloc[:test_valid.shape[0]//2]
test = shuffled.iloc[test_valid.shape[0]//2:]
print(test.sex.value_counts(normalize=True))
print(valid.sex.value_counts(normalize=True))
print(valid.shape[0])
print(test.shape[0])
valid.to_csv("results/validation_set.csv")
test.to_csv("results/test_set.csv")

male      0.88053
female    0.11947
Name: sex, dtype: float64
male      0.88026
female    0.11974
Name: sex, dtype: float64
1235921
1235922


In [45]:
train.sample(n=1000000).to_csv("results/train_subset.csv")

# Embeddings

In [ ]:
#df_train = pd.read_csv('train_set.csv')
import time
from gensim.models import Word2Vec
from tqdm import tqdm
from ast import literal_eval
total['processed'] = total['processed'].map(literal_eval)

tqdm.pandas()

start_time = time.time()

model = Word2Vec(sentences=total['processed'], 
                 sg=1, 
                 size=100,  
                 workers=4)

print(f'Time taken : {(time.time() - start_time) / 60:.2f} mins')

In [47]:
model.wv.save_word2vec_format('100d_embeds.txt')

In [46]:
len(model.wv.vocab.keys())

109692

In [25]:
list(model.wv.vocab.keys())[:5]

['', 'name', 'goes', 'to', 'ufc']

In [48]:
model.wv.most_similar('')


[('ftfy', 0.7760351300239563),
 ('smh', 0.678533136844635),
 ('smdh', 0.6585896611213684),
 ('amirite', 0.635604977607727),
 ('nope', 0.6278433203697205),
 ('jfc', 0.6257655620574951),
 ('ffs', 0.6208240985870361),
 ('whatttt', 0.6204562187194824),
 ('urlurl', 0.6194939613342285),
 ('lol', 0.6182695031166077)]

# Original code to process text

In [73]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub("\n", " ", s)
    s = re.sub(r"([.!?])", r" \1 ", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    #s = re.sub(r"[\d]+", "NUM", s)
    return s

In [38]:
class Lang:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        if len(sentence.split(' ')) < 4:
            return
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addSentenceNew(self,lst):
        if len(lst) < 4:
            return
        for word in lst:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
def createDic(df):
    df['text'] = df.body.map(normalizeString)
    input = Lang()
    print("Counting words...")
    for dp in df.body:
        input.addSentence(dp)
    print("Counted words:")
    print(input.n_words)
    return input

In [57]:
new[new.processed.isna()]

,body,sex,NEL,subreddit,text,processed
1424871,something like 30-40% of latinos vote republi...,male,Q22686,politics,something like of latinos vote republicansame ...,NaN
1424872,[NAME] is your friend,male,Q312015,politics,name is your friend,NaN
1424873,a belief that the 1987 railroad track deaths ...,male,Q1124,politics,a belief that the railroad track deaths of kev...,NaN
1424874,[NAME] really is the GOAT,male,Q22686,The_Donald,name really is the goat,NaN
1424875,Literally Scooby-Doo level shit right here.\n\...,female,Q6294,The_Donald,literally scooby doo level shit right here . n...,NaN
...,...,...,...,...,...,...
8171121,Labour voted against most of the deals that wh...,male,Q291169,unitedkingdom,labour voted against most of the deals that wh...,NaN
8171122,Didnt a report come out about the FBI bias jus...,male,Q22212,politics,didnt a report come out about the fbi bias jus...,NaN
8171123,"Well, you know how media coverage works...we h...",male,Q22686,canada,well you know how media coverage works . . . w...,NaN
8171124,I do not want to see [NAME] win the nomination...,male,Q785842,politics,i do not want to see name win the nomination b...,NaN


In [54]:
# Substitute overtly gendered words
new['processed'] = np.nan
for i in range(new.shape[0]):
    words = []
    for word in new.text.iloc[i].split(' '):
        # Replace OBVIOUSLY GENDERED words
        if word in subs.keys():
            words.append(subs[word])
        else:
            words.append(word)
    new['processed'].iloc[i] = words

<ipython-input-54-295f75d2c217>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['processed'] = np.nan
C:\Users\vanki\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\vanki\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

KeyboardInterrupt: 

In [ ]:
#Recount words
lang = createDic(new)
#originally 500,000ish

In [ ]:
NUM_OCC = 50
c=0
for word in lang.word2index:
    if lang.word2count[word] > 50:
        c+=1
print(c)
#originally 111000ish

In [ ]:
def createNewDic(df):
    input = Lang()
    print("Counting words...")
    for dp in df.processed:
        input.addSentenceNew(dp)
    print("Counted words:")
    print(input.n_words)
    return input
lang_new = createNewDic(new)

In [ ]:
NUM_OCC = 50
c=0
for word in lang_new.word2index:
    if lang_new.word2count[word] > 50:
        c+=1
print(c)
#originally 111000ish

In [ ]:
# Remove comments <4 words
new['len'] = new.processed.map(len)
new2 = new[new['len']> 4]
del new
print(new2.shape)

In [ ]:
# Create one with only words than appear >50 words (so we have two columns. One can be combined with pre-made gLove vectors!)
new2['processed_sub100000'] = ''
for i in range(new2.shape[0]):
    words = []
    for word in new2.processed:
        if lang_new.word2count[word] < 50:
            words.append['unk']
        else:
            words.append[word]
    new2['processed_sub100000'].iloc[i] = words

In [ ]:
new2.to_csv("interim.csv")

In [26]:
new2.NEL.value_counts()[:17].sum()

5718148

In [27]:
new2.NEL.value_counts()[17:].sum()

2452977

In [ ]:
# Divide train/valid/test set!
# How could I divide this to test cross-partisan?

In [19]:
new2.shape[0]*0.7

5719788.199999999

In [20]:
import pandas as pd
df = pd.read_csv("results/total.csv")
df.columns

C:\Users\vanki\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['body', 'sex', 'NEL', 'subreddit', 'text', 'processed', 'len',
       'processed_sub100000', 'processed_subsmaller'],
      dtype='object')

In [21]:
df = df[~df.subreddit.str.startswith('Q')]

In [22]:
df = df[df.sex.isin(['male','female'])]

In [23]:
df.len = df.len.map(float)

In [24]:
df.len.mean()

51.27509637849603

In [25]:
df.len.std()

65.43451905738064